# 德国快速动员进攻模型

## 导入包

In [1]:
import mesa
import mesa_geo as mg
import mesa_geo.visualization as mgv
from shapely.geometry import Point

d:\anaconda\envs\CSSforthinking\Lib\site-packages\solara\validate_hooks.py:122: UserWarning: d:\anaconda\envs\CSSforthinking\Lib\site-packages\mesa\experimental\jupyter_viz.py:163: JupyterViz: `use_state` found within a nested function created on line 155
To suppress this check, replace the line with:
        grid_layout, set_grid_layout = solara.use_state(grid_layout_initial)  # noqa: SH104

Make sure you understand the consequences of this, by reading about the rules of hooks at:
    https://solara.dev/documentation/advanced/understanding/rules-of-hooks

  warnings.warn(str(e))


## 德国进攻方的agent class

In [2]:
class german_army(mg.GeoAgent):
    def __init__(self, unique_id, model, geometry, crs ,attack_value, attack_speed, soldier_number,enermy_distance, wartype="prepared"):
        super().__init__(unique_id, model, geometry, crs)
        self.breed="german_army"
        self.attack_value=attack_value
        self.attack_speed=attack_speed
        self.soldier_number=soldier_number
        self.enermy_distance=enermy_distance
        self.wartype= wartype
        self.set_place=0

    def __repr__(self): 
        return "german_army " + str(self.unique_id)
    
    def calculate_distance(self,x):
        return self.model.space.distance(self, x)
    
    def step(self):
        if self.wartype=="prepared":
        #见到法国军队，德国agent变为战争状态，往他们的位置移动。
            neighbors=self.model.space.get_neighbors_within_distance(self,self.enermy_distance)
            neighbors_setplace=self.model.space.get_neighbors_within_distance(self,2000000)
            enermy_army_neighbors=[]
            for agent in neighbors:
                if agent is not None and agent.breed=="France_army" and agent.wartype!="dead":
                    enermy_army_neighbors.append(agent)
            if enermy_army_neighbors is None:
                #前往最近的据点agent
                set_neighbors=[]
                for agent in neighbors_setplace:
                    if agent.wartype=="set_a_place":
                        set_neighbors.append(agent)
                closest_set = min(set_neighbors, key=lambda x: self.calculate_distance(x))
            #移动到这个点的临近位置：
                x0=closest_set.geometry.x
                y0=closest_set.geometry.y
                x_moveto = x0 + self.random.randint(100,200)
                y_moveto = y0 + self.random.random(100,200)
                self.geometry=Point(x_moveto,y_moveto)

            else:
                if enermy_army_neighbors:
            #选择一个最近的army，去它的周围;此处用了min+lamda函数的组合
                    closest_enermy = min(enermy_army_neighbors, key=lambda x: self.calculate_distance(x))
                #移动到这个点的临近位置：
                    x0=closest_enermy.geometry.x
                    y0=closest_enermy.geometry.y
                    x_moveto = x0 + self.random.randint(100,200)
                    y_moveto = y0 + self.random.randint(100,200)
                    self.geometry=Point(x_moveto,y_moveto)
                    #与法国军队对比数值，战胜的话就停留一个step设置据点（转变为据点状态）
                    if self.attack_speed > closest_enermy.attack_speed:
                        closest_enermy.soldier_number= closest_enermy.soldier_number-1.5*self.attack_value*self.soldier_number
                        self.soldier_number=self.soldier_number-self.attack_value*self.soldier_number
                    else:
                        closest_enermy.soldier_number= closest_enermy.soldier_number-0.8*self.attack_value*self.soldier_number
                        self.soldier_number=self.soldier_number-self.attack_value*self.soldier_number

                    if closest_enermy.soldier_number<=0 and self.soldier_number > 0:
                        closest_enermy.wartype="dead"
                        self.wartype="set_a_place"
                        self.set_place=self.set_place+1
                    elif closest_enermy.soldier_number<=0 and self.soldier_number <= 0:
                        self.wartype="dead"
                        closest_enermy.wartype="dead"
                    elif closest_enermy.soldier_number >0 and self.soldier_number <= 0:
                        self.wartype="dead"
                        closest_enermy.wartype="prepared"
                    elif closest_enermy.soldier_number >0 and self.soldier_number > 0:
                        closest_enermy.wartype="in_war"
                        self.wartype="in_war"
                
            
        elif self.wartype=="in_war":
            neighbors=self.model.space.get_neighbors_within_distance(self,500)
            fclosest_enermy=[neighbor for neighbor in neighbors if neighbor.breed=="France_army"]
            if fclosest_enermy:  
                closest_enermy = fclosest_enermy[0]  
            else:  
                closest_enermy = None  # 或者您可以选择的默认值            
            closest_enermy.soldier_number= closest_enermy.soldier_number-self.attack_value*self.soldier_number
            self.soldier_number=self.soldier_number-self.attack_value*self.soldier_number

            if closest_enermy.soldier_number<=0:
                closest_enermy.wartype="dead"
                self.wartype="set_a_place"
            elif closest_enermy.soldier_number >0 and self.soldier_number <= 0:
                self.wartype="dead"
                closest_enermy.wartype="prepared"
            elif closest_enermy.soldier_number >0 and self.soldier_number > 0:
                closest_enermy.wartype="in_war"
                self.wartype="in_war"  

        elif self.wartype=="set_a_place":
            self.set_place=self.set_place+1

            if self.set_place >=2:
                #在据点可以回血
                self.soldier_number=self.soldier_number*1.2
                self.wartype="prepared"


        
        
        self.model.counts[self.wartype] = self.model.counts[self.wartype]+1


## 法国防守方的agent

In [3]:
class france_army(mg.GeoAgent):
    def __init__(self, unique_id, model, geometry, crs,soldier_number,attack_speed):
        super().__init__(unique_id, model, geometry, crs)
        self.breed = "France_army"
        self.soldier_number=soldier_number
        self.attack_speed=attack_speed
        self.wartype= "prepared"

    def __repr__(self): 
        return "german_army " + str(self.unique_id)
    
    def step(self):
        #有新的援兵动员
        if self.wartype=="in_war":
            self.soldier_number=self.soldier_number*1.5
        elif self.wartype=="prepared":
            self.soldier_number=self.soldier_number
        
        self.model.counts[self.wartype] += 1


## 设置边界地区的地块

In [4]:
class regionAgent(mg.GeoAgent):
    def __init__(self,unique_id, model, geometry, crs,wartype="not in war"):
        super().__init__(unique_id, model, geometry, crs)
        self.breed="region"

        self.wartype=wartype
    
    def __repr__(self):
        return "Neighbourhood " + str(self.unique_id)


    def hotspot(self):
        neighbors = self.model.space.get_intersecting_agents(self)
        waragent = [
            agent for agent in neighbors if agent.wartype == "in_war"or agent.wartype=="set_a_place"
        ]
        if waragent:
            self.wartype="region in war"
        else:
            self.wartype="not in war"

    def step(self):
        self.hotspot()
        self.model.counts[self.wartype] += 1
        
    

In [5]:
def get_in_war_count(model):
    return model.counts["in_war"]


def get_set_a_place_count(model):
    return model.counts["set_a_place"]


def get_prepared_count(model):
    return model.counts["prepared"]


def get_dead_count(model):
    return model.counts["dead"]

def get_region_in_war_count(model): 
    return model.counts["region in war"]

def get_region_not_in_war(model):
    return model.counts["not in war"]

## 设计交战模型

In [6]:
class warmodel(mesa.Model):

    geojson_regions = "D:\编程的文档\python\mesa-geo库的学习\german.geojson"
    unique_id = "regionnum"

    def __init__(self,german_army_size=60,france_army_size=60):
        super().__init__()
        self.german_army_size=german_army_size
        self.france_army_size=france_army_size

        self.schedule = mesa.time.RandomActivationByType(self)
        self.space = mg.GeoSpace(warn_crs_conversion=False)

        self.counts = None #added
        self.reset_counts() #added

        self.datacollector = mesa.DataCollector(
            {
                "in_war": get_in_war_count,
                "set_a_place":get_set_a_place_count,
                "prepared": get_prepared_count,
                "dead": get_dead_count,
                "region in war":get_region_in_war_count,
                "not in war":get_region_not_in_war

            }
        )

        ac = mg.AgentCreator(regionAgent, model=self)

        region_agents=ac.from_file(
            self.geojson_regions, unique_id=self.unique_id
        )           
        
        self.space.add_agents(region_agents)
        for agent in region_agents:
            self.schedule.add(agent)

    #根据region是德国还是法国分别将代理放进去

        for i in range(german_army_size):
            attack_value=0.6 + self.random.random() * 0.5 
            attack_speed= self.random.randint(8,20)
            soldier_number=self.random.randint(3000,6000)
            enermy_distance=90000

            unique_german_army = mg.AgentCreator(
            german_army,
            model=self,
            crs=self.space.crs,
            agent_kwargs={"attack_value": attack_value, 
             "attack_speed":attack_speed,
             "soldier_number":soldier_number,
             "enermy_distance": enermy_distance
             }
            )

            x_home, y_home = self.find_home(region_agents,"german")
            #注意到这里有unique_person.create_agent,我们可以根据geometry and id通过creare_agent生成agent
            german_army_agent = unique_german_army.create_agent(
                Point(x_home, y_home), "G" + str(i), 
            )
            self.space.add_agents(german_army_agent)
            self.schedule.add(german_army_agent)   

        for i in range(france_army_size):

            attack_speed= self.random.randint(6,10)
            soldier_number=self.random.randint(4000,8000)

            unique_german_army = mg.AgentCreator(
            france_army,
            model=self,
            crs=self.space.crs,
            agent_kwargs={
             "attack_speed":attack_speed,
             "soldier_number":soldier_number
             }
            )

            x_home, y_home = self.find_home(region_agents,"france")
            #注意到这里有unique_person.create_agent,我们可以根据geometry and id通过creare_agent生成agent
            france_army_agent = unique_german_army.create_agent(
                Point(x_home, y_home), "F" + str(i), 
            )
            self.space.add_agents(france_army_agent)
            self.schedule.add(france_army_agent)  

        
    
    def find_home(self,region_agents,country):
        self.country=country
        select_in_germanregions=[]
        select_in_franceregions=[]
        for agent in region_agents:
            if "g" in agent.unique_id:
                select_in_germanregions.append(agent)
            elif "f" in agent.unique_id:
                select_in_franceregions.append(agent)
        
        if self.country=="german":
            this_region = self.random.randint(
                0, len(select_in_germanregions) - 1
            )  # Region where agent starts
            center_x, center_y = select_in_germanregions[
                this_region
            ].geometry.centroid.coords.xy
            this_bounds = select_in_germanregions[this_region].geometry.bounds
            spread_x = int(
                this_bounds[2] - this_bounds[0]
            )  # Heuristic for agent spread in region
            spread_y = int(this_bounds[3] - this_bounds[1])
            this_x = center_x[0] + self.random.randint(0, spread_x) - spread_x / 2
            this_y = center_y[0] + self.random.randint(0, spread_y) - spread_y / 2
            return this_x,this_y
        
        elif self.country=="france":
            this_region = self.random.randint(
                0, len(select_in_franceregions) - 1
            )  # Region where agent starts
            center_x, center_y = select_in_franceregions[
                this_region
            ].geometry.centroid.coords.xy
            this_bounds = select_in_franceregions[this_region].geometry.bounds
            spread_x = int(
                this_bounds[2] - this_bounds[0]
            )  # Heuristic for agent spread in region
            spread_y = int(this_bounds[3] - this_bounds[1])
            this_x = center_x[0] + self.random.randint(0, spread_x) - spread_x / 2
            this_y = center_y[0] + self.random.randint(0, spread_y) - spread_y / 2
            return this_x,this_y
        
    def reset_counts(self):
        self.counts = {
            "in_war": 0,
            "set_a_place": 0,
            "prepared": 0,
            "dead": 0,
            "region in war":0,
            "not in war":0
            }

    def step(self):

        self.reset_counts() 
        self.schedule.step()
        self.datacollector.collect(self) 


In [7]:
model = warmodel()
for i in range(10): 
    model.step()

## 可视化部分

In [8]:
model.datacollector.get_model_vars_dataframe()

,in_war,set_a_place,prepared,dead,region in war,not in war
0,18,31,18,53,6,1
1,3,15,49,53,6,1
2,34,18,11,57,5,2
3,24,10,29,57,5,2
4,38,6,17,59,2,5
5,42,6,13,59,2,5
6,49,0,12,59,2,5
7,55,0,6,59,2,5
8,55,0,6,59,2,5
9,55,0,6,59,2,5


In [9]:
def warmodel_draw(agent):
    """
    Portrayal Method for canvas
    """

    portrayal = {}
    if isinstance(agent, german_army): 
        if agent.wartype == "in_war":
            portrayal["color"] = "red"
        elif agent.wartype == "set_a_place":
            portrayal["color"] = "yellow"
        elif agent.wartype == "prepared":
            portrayal["color"] = "Blue"
        else:
            portrayal["color"] = "grey"
            
    if isinstance(agent, france_army): 
        if agent.wartype == "in_war":
            portrayal["color"] = "pink"
        elif agent.wartype == "prepared":
            portrayal["color"] = "green"
        else: 
            portrayal["color"] = "black"
            
    if isinstance(agent, regionAgent):
        if agent.wartype == "region in war":
            portrayal["color"] = "Red"
        else: 
            portrayal["color"] = "green"
    
    return portrayal


model_params = {
    "german_army_size": {
        "type": "SliderInt",
        "value": 60,
        "label": "german_army_size",
        "min": 30,
        "max": 100, 
        "step": 1,
    },
     "france_army_size": {
        "type": "SliderInt",
        "value": 60,
        "label": "france_army_size",
        "min": 30,
        "max": 100, 
        "step": 1,
     }}

In [10]:
page = mgv.GeoJupyterViz(
    warmodel,
    model_params,
    measures= [["in_war", "prepared", "set_a_place", "dead"], ["region in war", "not in war"]],
    name="warmodel_german-france",   #界面名
    agent_portrayal=warmodel_draw,  
    zoom=10,
    scroll_wheel_zoom=False
)
# This is required to render the visualization in the Jupyter notebook
page

Cannot show ipywidgets in text